In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn import linear_model
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsRegressor
from KCrossVal import kFoldCross, MeanAbsError, BinaryAccuracy
import time

In [32]:
dataset_df = pd.read_csv('./data/diabetes.csv')
matrix = dataset_df.to_numpy()
X = matrix[:,:-1]
Y = matrix[:,-1]
for c in range(X.shape[1]):
    X[:,c] = X[:,c] - X[:,c].mean()
    X[:,c] = X[:,c] / X[:,c].std()
dataset_df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [42]:
C_values = [0.1,0.5,1]
results = []
for C in C_values:
    reg = linear_model.LogisticRegression(C=C)
    res = kFoldCross(reg.fit, reg.predict, X, Y, 5, MeanAbsError)
    res = C, res[0], res[1], res[3]
    results.append(res)

df = pd.DataFrame(results, columns=['C', 'ValErr_mean', 'ValErr_std', 'seconds'])
df.sort_values(by='ValErr_mean')

,C,ValErr_mean,ValErr_std,seconds
1,0.5,0.227791,0.034725,0.022888
2,1.0,0.227791,0.033236,0.026360
0,0.1,0.231704,0.037038,0.042001


In [46]:
for i, c in enumerate(dataset_df.columns):
     print(c)
     print(reg.coef_.ravel()[i])

Pregnancies
0.4073710677188941
Glucose
1.0350959512237818
BloodPressure
-0.19951543910203834
SkinThickness
-0.040224502305011324
Insulin
-0.09076190160798953
BMI
0.7951451117527885
DiabetesPedigreeFunction
0.347298348341786
Age
0.11681063593158722
Outcome


IndexError: index 8 is out of bounds for axis 0 with size 8

In [37]:
reg.predict(X).mean()

0.2721354166666667

In [38]:
Y.mean()

0.3489583333333333

In [39]:
alpha = [1,10,20,30]
results = []
for a in alpha:
    reg = linear_model.Ridge(alpha=a)
    res = kFoldCross(reg.fit, reg.predict, X, Y-0.5, 5, BinaryAccuracy)
    res = a, res[0], res[1], res[3]
    results.append(res)

df = pd.DataFrame(results, columns=['alpha', 'ValErr_mean', 'ValErr_std', 'seconds'])
df.sort_values(by='ValErr_mean')

,alpha,ValErr_mean,ValErr_std,seconds
3,30,0.226500,0.033165,0.007780
1,10,0.227799,0.035024,0.007629
2,20,0.227799,0.035024,0.006620
0,1,0.230396,0.036772,0.012114


In [40]:
(reg.predict(X) > 0).astype(int).mean()

0.2565104166666667